In [3]:
import tensorflow as tf
from multiprocessing import Process
from time import sleep
import time
import numpy as np
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble.forest import RandomForestClassifier
from kfoldwrapper import TFKFoldWrapper, SKKFoldWrapper, RFKFoldWrapper, evaluate_performance
from load_adult import load_adult
from load_yeast import load_yeast
from load_letter import load_letter

Using TensorFlow backend.


In [4]:
# X, y = sklearn.datasets.load_digits(return_X_y=True)
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# x_train = x_train.astype(np.float32)
# x_test = x_test.astype(np.float32)
# print(x_train.shape, x_test.shape)

x_train, x_test, y_train, y_test = load_letter()
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)
print("Num Classes: {}".format(len(set(y_test))))

((16000, 16), (4000, 16))
((16000,), (4000,))
Num Classes: 26


In [5]:
n_train = 16000   # 1038      # 32561
n_test = 4000     # 446       # 16281
num_features = x_test.shape[-1]
if len(y_test.shape) == 1:
    num_classes = len(set(y_test))
else:
    num_classes = y_test.shape[-1]
n_estimators = 50
max_leaf_nodes = 1000

In [11]:
number_workers = 2
worker_ips = []
for i in range(number_workers):
    worker_ips.append("localhost:333{}".format(i))

In [12]:
cluster = tf.train.ClusterSpec({
    "worker": worker_ips,
    "ps": [
        "localhost:3376"
    ]
})

In [13]:
# Worker side
def get_prob(data_type="train", idx=0):
    with tf.variable_scope("prob_{}".format(data_type), reuse=True):
        x = tf.get_variable(name='x_{}'.format(idx),
                            initializer=tf.zeros((594, 10), dtype=np.float32),
                            dtype=np.float32)
    return x

In [14]:
def parameter_server(num_workers=2,
                     n_train=32561,
                     n_test=16281,
                     num_features=113,
                     num_classes=2):
    with tf.device("/job:ps/task:0"):
        cont = tf.Variable([True for i in range(num_workers)], name='continue')
        shut_down = tf.Variable(False, name='shutdown')
        concat = tf.Variable(False, name='concat')
        layer_train_prob = tf.Variable(np.zeros((n_train, num_classes)), name='layer_train_prob')
        layer_test_prob = tf.Variable(np.zeros((n_test, num_classes)), name='layer_test_prob')
        ready = tf.Variable([False for _ in range(num_workers)], name='ready')
        prob_train_xs = [None for _ in range(num_workers)]
        with tf.variable_scope("prob_train"):
            for i in range(num_workers):
                prob_train_xs[i] = tf.get_variable(name='x_{}'.format(i),
                                                   initializer=tf.zeros((n_train, num_classes), dtype=np.float32),
                                                   dtype=np.float32)
        prob_test_xs = [None for _ in range(num_workers)]
        with tf.variable_scope("prob_test"):
            for i in range(num_workers):
                prob_test_xs[i] = tf.get_variable(name='x_{}'.format(i),
                                                  initializer=tf.zeros((n_test, num_classes), dtype=np.float32),
                                                  dtype=np.float32)

    server = tf.train.Server(cluster,
                             job_name="ps",
                             task_index=0)
    sess = tf.Session(target=server.target)

    print("Parameter server: waiting for cluster connection...")
    sess.run(tf.report_uninitialized_variables())
    print("Parameter server: cluster ready!")

    print("Parameter server: initializing variables...")
    sess.run(tf.global_variables_initializer())
    print("Parameter server: variables initialized")
    
    train_acc_records = []
    test_acc_records = []
    best_test_acc = 0.0
    best_layer = 0
    early_stopping = False

    num_layer = 0
    while True:
        while not sess.run(ready).all():
            ready_value = sess.run(ready)
            # print("ready indicator: ", ready_value)
            sleep(1.0)
        print("All Ready!!!")

#         for i in range(2):
#             print("prob_train_xs[{}] = {}".format(i, sess.run(prob_train_xs[i])))
#             print("prob_test_xs[{}] = {}".format(i, sess.run(prob_test_xs[i])))

        # print("Layer train Probability: {}".format(sess.run(layer_train_prob)))
        # print("Layer test  Probability: {}".format(sess.run(layer_test_prob)))
        train_acc = evaluate_performance(sess.run(layer_train_prob), y_train)
        test_acc = evaluate_performance(sess.run(layer_test_prob), y_test)
        print("LAYER train={}, test={}".format(train_acc, test_acc))
        train_acc_records.append(train_acc)
        test_acc_records.append(test_acc)
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_layer = num_layer
        if num_layer - best_layer >= 4:
            early_stopping = True
        each_train_acc = [evaluate_performance(sess.run(prob_train_xs[i]), y_train) for i in range(num_workers)]
        each_test_acc = [evaluate_performance(sess.run(prob_test_xs[i]), y_test) for i in range(num_workers)]
        print("Every train acc = {}".format(each_train_acc))
        print("Every test  acc = {}".format(each_test_acc))
        num_layer += 1
        sess.run(ready.assign([False for i in range(num_workers)]))
        if num_layer > 0:     # to concat
            sess.run(concat.assign(True))
        if early_stopping or num_layer > 30:             # judge earlystopping, or max_layer encountered
            print("Early Stopped, best_acc = {}, best_layer = {}".format(best_test_acc, best_layer))
            sess.run(shut_down.assign(True))
            break
        sess.run(cont.assign([True for i in range(num_workers)]))
        print("cont = {}".format(sess.run(cont)))
        print("Continue Training Layer {} ......".format(num_layer))

    print("Parameter server: blocking...")
    server.join()

In [15]:
def worker(num_workers,
           worker_n,
           n_train=32561,
           n_test=16281,
           num_features=113,
           num_classes=2):
    with tf.device("/job:ps/task:0"):
        cont = tf.Variable([True for i in range(num_workers)], name='continue')
        shut_down = tf.Variable(False, name='shutdown')
        concat = tf.Variable(False, name='concat')
        layer_train_prob = tf.Variable(np.zeros((n_train, num_classes)), name='layer_train_prob')
        layer_test_prob = tf.Variable(np.zeros((n_test, num_classes)), name='layer_test_prob')
        ready = tf.Variable([False for _ in range(num_workers)], name='ready')
        prob_train_xs = [None for _ in range(num_workers)]
        with tf.variable_scope("prob_train"):
            for i in range(num_workers):
                prob_train_xs[i] = tf.get_variable(name='x_{}'.format(i),
                                                   initializer=tf.zeros((n_train, num_classes), dtype=np.float32),
                                                   dtype=np.float32)
        prob_test_xs = [None for _ in range(num_workers)]
        with tf.variable_scope("prob_test"):
            for i in range(num_workers):
                prob_test_xs[i] = tf.get_variable(name='x_{}'.format(i),
                                                  initializer=tf.zeros((n_test, num_classes), dtype=np.float32),
                                                  dtype=np.float32)

    server = tf.train.Server(cluster,
                             job_name="worker",
                             task_index=worker_n)
    sess = tf.Session(target=server.target)

    print("Worker %d: waiting for cluster connection..." % worker_n)
    sess.run(tf.report_uninitialized_variables())
    print("Worker %d: cluster ready!" % worker_n)

    while sess.run(tf.report_uninitialized_variables()).any():
        print("Worker %d: waiting for variable initialization..." % worker_n)
        sleep(1.0)
    print("Worker %d: variables initialized" % worker_n)
    
    num_layer = 0
    while True:
        print("[Worker {}] Start training Layer {} ......".format(worker_n, num_layer))

        # now continue, close continue tag ============================================
        indices = tf.constant([[worker_n],])
        updates = tf.constant([False])
        set_cont = cont.scatter_nd_update(indices, updates)
        sess.run(set_cont)
        # =============================================================================
        
        # concat ======================================================================
        need_concat = sess.run(concat)
        print("Need Concat ? {}".format(need_concat))
        if need_concat:
            true_x_train = np.hstack((x_train, ) + tuple(sess.run(prob_train_xs[i]) for i in range(num_workers)))
            true_x_test = np.hstack((x_test, ) + tuple(sess.run(prob_test_xs[i]) for i in range(num_workers)))
        else:
            true_x_train = x_train
            true_x_test = x_test
        print("true_x_train, true_x_test shape = {}, {}".format(true_x_train.shape, true_x_test.shape))
        print("true_x_train = {}".format(true_x_train))
        # end concat ==================================================================

        est_name = 'layer-{}-estimator-{}-{}folds'.format(num_layer, worker_n, 3)
        seed = (42 + hash("[estimator] {}".format(est_name))) % 1000000007

        # Tensorflow KFoldWrapper =====================================================
        est_args = {'num_classes': num_classes, 'num_features': true_x_train.shape[-1], 'regression': False,
                    'num_trees': n_estimators, 'max_nodes': max_leaf_nodes,
                    'base_random_seed': None}
        from kfoldwrapper import TFKFoldWrapper
        kfw = TFKFoldWrapper
        # =============================================================================

        # SKlearn KFoldWrapper ========================================================
#         est_args = {'n_estimators': n_estimators, 'max_features': 'auto', 'max_leaf_nodes': max_leaf_nodes,
#                     'random_state': None}
        
#         from kfoldwrapper import TFKFoldWrapper, SKKFoldWrapper, RFKFoldWrapper
#         if worker_n < number_workers/2:
#             kfw = SKKFoldWrapper
#         else:
#             kfw = RFKFoldWrapper
        # =============================================================================

        y_proba_train, y_proba_test = kfw(name=est_name, n_folds=3,
                                          task='classification',
                                          seed=seed, dtype=np.float32,
                                          est_args=est_args, cv_seed=seed).fit_transform(
            X=true_x_train, y=y_train, x_test=true_x_test, y_test=y_test)
        # =============================================================================

        x = get_prob("test", worker_n)
        # print("x.name = {}".format(x.name))
        sess.run(x.assign(y_proba_test))
        sess.run(layer_test_prob.assign_add(y_proba_test))
        # ==============================================================================

        x = get_prob("train", worker_n)
        # print("x.name = {}".format(x.name))
        sess.run(x.assign(y_proba_train))
        sess.run(layer_train_prob.assign_add(y_proba_train))
        # ==============================================================================

        # ready ========================================================================
        indices = tf.constant([[worker_n],])
        updates = tf.constant([True])
        set_ready = ready.scatter_nd_update(indices, updates)
        sess.run(set_ready)
        # ==============================================================================
        
        # wait the indicator of the next run ===========================================
        print("[Worker {}] wait the indicator of the next run ...".format(worker_n))
        is_shutdown = False
        while not sess.run(cont)[worker_n]:  # while not continue
            time.sleep(1.0)
            print("cont indicator: ", sess.run(cont)[worker_n])
            if sess.run(shut_down):
                is_shutdown = True
                break
        if is_shutdown:
            print("Worker %d shutting down..." % worker_n)
            break
        num_layer += 1

    print("Worker %d: blocking..." % worker_n)
    server.join()


In [16]:
ps_proc = Process(target=parameter_server, args=(number_workers, n_train, n_test, num_features, num_classes, ), daemon=True)
w_procs = []
for i in range(number_workers):
    w_proc = Process(target=worker,
                     args=(number_workers, i, n_train, n_test, num_features, num_classes, ),
                     daemon=True)
    w_procs.append(w_proc)
    # w2_proc = Process(target=worker, args=(2, 1, n_train, n_test, num_features, num_classes, ), daemon=True)

TypeError: __init__() got an unexpected keyword argument 'daemon'

In [29]:
ps_proc.start()

Parameter server: waiting for cluster connection...
Parameter server: cluster ready!
Parameter server: initializing variables...
Parameter server: variables initialized


In [30]:
for i in range(number_workers):
    w_procs[i].start()

Worker 1: waiting for cluster connection...
Worker 2: waiting for cluster connection...
Worker 3: waiting for cluster connection...
Worker 0: waiting for cluster connection...
Worker 4: waiting for cluster connection...
Worker 5: waiting for cluster connection...
Worker 6: waiting for cluster connection...
Worker 7: waiting for cluster connection...
Worker 0: cluster ready!
Worker 5: cluster ready!
Worker 1: cluster ready!
Worker 3: cluster ready!
Worker 2: cluster ready!
Worker 4: cluster ready!
Worker 6: cluster ready!
Worker 7: cluster ready!
Worker 5: waiting for variable initialization...
Worker 0: waiting for variable initialization...
Worker 3: waiting for variable initialization...
Worker 1: waiting for variable initialization...
Worker 2: waiting for variable initialization...
Worker 4: waiting for variable initialization...
Worker 6: waiting for variable initialization...
Worker 7: waiting for variable initialization...
Worker 5: variables initialized
[Worker 5] Start trainin

  est = Estimator(...) -> est = SKCompat(Estimator(...))WARNING:tensorflow:From /home/huqiu/SPARE/DeepForestOnTF/kfoldwrapper.py:161: calling BaseEstimator.fit (from tensorflow.contrib.learn.python.learn.estimators.estimator) with x is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate clas

Please use tensorflow/transform or tf.data.

Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
Instructions for updating:
Please feed input to tf.data to support dask.
INFO:tensorflow:Using default config.
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
Instructions for updating:
Please feed input to tf.data to support dask.WARNING:tensorflow:From /home/huqiu/programs/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/learn_io/data_feeder.py:96: extract_dask_data (from tensorflow.contrib.learn.python.learn.learn_io.dask_io) is deprecated and will be removed in a future version.
Instructions for updating:
Please feed input to tf.data to support dask.WARNING:tensorflow:Using temporary folder as model directory: /tmp/tmpfi

  est = Estimator(...) -> est = SKCompat(Estimator(...))WARNING:tensorflow:From /home/huqiu/programs/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/learn_io/data_feeder.py:100: extract_pandas_data (from tensorflow.contrib.learn.python.learn.learn_io.pandas_io) is deprecated and will be removed in a future version.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please access pandas data directly.WARNING:tensorflow:From /home/huqiu/SPARE/DeepForestOnTF/kfoldwrapper.py:161: calling BaseEstimator.fit (from tensorflow.contrib.learn.python.learn.estimators.estimator) with y is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...

Please convert numpy dtypes explicitly.WARNING:tensorflow:From /home/huqiu/programs/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/estimators/estimator.py:143: setup_train_data_feeder (from tensorflow.contrib.learn.python.learn.learn_io.data_feeder) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tensorflow/transform or tf.data.

Instructions for updating:
Please use tensorflow/transform or tf.data.
INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_trees': 50, 'max_nodes': 1000, 'bagging_fraction': 1.0, 'feature_bagging_fraction': 1.0, 'num_splits_to_consider': 10, 'max_fertile_nodes': 0, 'split_after_samples': 250, 'valid_leaf_threshold': 1, 'dominate_method': 'bootstrap', 'dominate_fraction': 0.99, 'model_name': 'all_dense', 'split_finish_name': 'basic', 'split_pruning_name': 'none', 'collate_examples': False, 'checkpoint_stats': False, 'use_running_stats_method': False, 'initialize_avera

...

Process Process-38:
Traceback (most recent call last):
  File "/home/huqiu/programs/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/huqiu/programs/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-27-c2491e70b36f>", line 90, in worker
    y_proba_train, y_proba_test = kfw(name=est_name, n_folds=3,
NameError: name 'kfw' is not defined


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmprqhgeip1/model.ckpt-127
Accuracy(layer-0-estimator-5-3folds - train_avg) = 90.4375%
Accuracy(layer-0-estimator-5-3folds - test_avg) = 90.7250%


Process Process-43:
Traceback (most recent call last):
  File "/home/huqiu/programs/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/huqiu/programs/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-27-c2491e70b36f>", line 90, in worker
    y_proba_train, y_proba_test = kfw(name=est_name, n_folds=3,
NameError: name 'kfw' is not defined


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Accuracy(layer-0-estimator-4-3folds - train_avg) = 90.8937%
Accuracy(layer-0-estimator-4-3folds - test_avg) = 91.2500%


Process Process-42:
Traceback (most recent call last):
  File "/home/huqiu/programs/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/huqiu/programs/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-27-c2491e70b36f>", line 90, in worker
    y_proba_train, y_proba_test = kfw(name=est_name, n_folds=3,
NameError: name 'kfw' is not defined


In [31]:
for proc in [ps_proc, ] + w_procs:
    proc.terminate()